In [2]:
import tensorflow.keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.utils import to_categorical

import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
import random

from tqdm.notebook import tqdm as log_progress

In [8]:
#load and clean data
MODEL_NAME = "Test"
#MODEL_NAME = 'shakespear2.0'
data_file_name = "border-tales.epub.txt"
SEQUENCE_LENGTH = 5 #how many items are fed into the ai per sequence

raw = open("data/" + data_file_name, "r", encoding='utf-8')

lines = []
print("Loading data...")
for line in log_progress(raw):
    if str(line) != '\n':
        lines.append(str(line).lower())

print("Tokenizing...")
#putting the entirety of the input data into one string
text = ""
for line in log_progress(lines):
    text += line
#tokenizer = RegexpTokenizer(r'[a-zA-Z]')
tokenizer = RegexpTokenizer(r'\w+')
tokenized = tokenizer.tokenize(text)

#converting the text into numbers to be processed by the embedding layer of the model
words = [] #one of each word in tokenized will be in here
filtering = lambda x : not x in words #for finding if the word should be added to the words array
find = lambda x : words.index(x) if x in words else len(words) #convert each word into a number. -1 means that the item isn't in the vocabulary

print("Finding every unique word...")
for word in log_progress(tokenized):
    if filtering(word):
        words.append(word)

#x data is every single word in the data set, in order
#y data is every single word that comes after the corresponding x value
x = []
y = []
print("Compiling dataset...")
counter = 0
for i in log_progress(range(int(len(tokenized)/SEQUENCE_LENGTH))):
    temp = []
    for j in range(0, SEQUENCE_LENGTH):
        if counter + j >= len(tokenized):
            while len(temp) < SEQUENCE_LENGTH:
                temp.append(0)
            break
        else:
            temp.append(find(tokenized[counter+j]))
    
    counter += SEQUENCE_LENGTH - 1
    x.append(temp)
    
    if counter+SEQUENCE_LENGTH < len(tokenized):
        y.append(find(tokenized[counter+SEQUENCE_LENGTH]))
    else:
        y.append(find(tokenized[0]))

x = np.asarray(x, np.int32)
y = np.asarray(y, np.int32)

x = np.reshape(x, (x.shape[0], SEQUENCE_LENGTH, 1))
y = to_categorical(y)

print(x.shape)
print(y.shape)

vocab_size = len(words)

Loading data...



Tokenizing...



Finding every unique word...



Compiling dataset...



(25828, 5, 1)
(25828, 8672)


In [12]:
#create and compile model
model = Sequential()

model.add(Embedding(vocab_size+1, 12, input_length=SEQUENCE_LENGTH)) #embedding: size of vocabulary, dimension of each value
model.add(LSTM(20, dropout=0.01))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(y.shape[1], activation='softmax'))

opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 5, 12)             118020    
_________________________________________________________________
lstm_3 (LSTM)                (None, 20)                2640      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                672       
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 8672)              286176    
Total params: 407,508
Trainable params: 407,508
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
#train model
hist = model.fit(x, y, epochs=200, verbose=1, batch_size=20, validation_split=0.01)
model.save(MODEL_NAME + ".h5")

plt.title("loss")
plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['val_loss'], label='val_loss')

Epoch 1/200
 670/1279 [==============>...............] - ETA: 7s - loss: 7.1417 - accuracy: 0.0738

KeyboardInterrupt: 

In [26]:
#test model
sample_length = 100
model = load_model("Test.h5")

#get input sentence and process
sentence = input("Enter first 10 words...").lower()
tokenizer = RegexpTokenizer(r'\w+')
root = tokenizer.tokenize(sentence)
root = [find(root[i]) for i in range(len(root))]

output = sentence + " "

for i in log_progress(range(sample_length)):
    tmp = np.array(root)
    tmp = np.reshape(tmp, (1, SEQUENCE_LENGTH, 1))
    pred = model.predict(tmp, verbose=0)
    next_word = pred.argmax()
    #next_word = np.random.choice(len(pred[0]), p=pred[0])
    output += words[next_word] + " "
    root.pop(0)
    root.append(next_word) #setting the next word

print(output)

Enter first 10 words...thy thee will kneel while walking down the road thee


  0%|          | 0/100 [00:00<?, ?it/s]

thy thee will kneel while walking down the road thee read since since muses since morn morn cruel past riches riches remain where nothing shape show do take cruel riches needs breed leave go tired praises after after every praise see shall as for when shadow being for heaven should nor be show by by shape by by by by but but shall shall though stay might myself scythe informer verse king king we how show in in in thy the time your will such such those those those nor when love is is a morrow head rank dear without ah calls delight so so at to my d i 
